# Import libaries

In [ ]:
# Ignore  the warnings
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

# data visualisation and manipulation
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns
from collections import Counter
 
#configure
# sets matplotlib to inline and displays graphs below the corressponding cell.
%matplotlib inline  
style.use('fivethirtyeight')
sns.set(style = 'whitegrid',color_codes = True)

#model selection
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score,precision_score,recall_score
from sklearn.metrics import confusion_matrix,roc_curve
from sklearn.metrics import roc_auc_score,precision_recall_curve,auc,f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder

#preprocess.
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import img_to_array, array_to_img, load_img
from sklearn.utils import resample
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE

#dl libraraies
from keras import backend as K
from tensorflow.keras import layers, Model
from keras.models import Sequential
from keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.preprocessing import image
from tensorflow.keras.optimizers import Adam,SGD,Adagrad,Adadelta,RMSprop
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.metrics import SparseCategoricalAccuracy
from keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
from tensorflow.keras import regularizers

# specifically for cnn
from keras.layers import Dropout, Flatten, Activation
from keras.layers import Conv2D, MaxPool2D, BatchNormalization
 
import tensorflow as tf
import random as rn

# specifically for manipulating zipped images and getting numpy arrays of pixel values of images.
import cv2                  
import numpy as np  
from tqdm import tqdm
import os                   
from random import shuffle  
from zipfile import ZipFile
from PIL import Image

from sklearn.metrics import classification_report
import itertools
from tensorflow import keras
import re

# Data Preprocessing

In [ ]:
(img_height, img_width) = (299, 299)
batch_size = 32
epochs = 40

In [ ]:
p1 = '/kaggle/input/seniortraindata/Project_Full/Project_Full/C_A1_Naildystrophy-20230129T112110Z-001/C_A1_Naildystrophy'
p2 = '/kaggle/input/seniortraindata/Project_Full/Project_Full/C_A2_Naildystrophy-20230129T111929Z-001/C_A2_Naildystrophy'
p3 = '/kaggle/input/seniortraindata/Project_Full/Project_Full/C_A1_Onychomycosis-20230129T112107Z-001/C_A1_Onychomycosis'
p4 = '/kaggle/input/seniortraindata/Project_Full/Project_Full/C_A2_Onychomycosis-20230129T111926Z-001/C_A2_Onychomycosis'
path1 = pd.Series(os.listdir(p1))
path2 = pd.Series(os.listdir(p2))
path3 = pd.Series(os.listdir(p3))
path4 = pd.Series(os.listdir(p4))

df1 = pd.DataFrame(path1, columns = ['Image_name'])
df1['Image_name'] = p1 + '/' + df1['Image_name'] 
df2 = pd.DataFrame(path2, columns = ['Image_name'])
df2['Image_name'] = p2 + '/' + df2['Image_name'] 
df3 = pd.DataFrame(path3, columns = ['Image_name'])
df3['Image_name'] = p3 + '/' + df3['Image_name'] 
df4 = pd.DataFrame(path4, columns = ['Image_name'])
df4['Image_name'] = p4 + '/' + df4['Image_name'] 

df = pd.concat([df1, df2, df3, df4])
df['Label'] = np.where(df['Image_name'].str.contains(pat = 'Naildystrophy'), '0', '1')

In [ ]:
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.applications.nasnet import NASNetLarge
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2
from tensorflow.keras.applications.densenet import DenseNet121, DenseNet201
from tensorflow.keras.applications.mobilenet import MobileNet
#from tensorflow.keras.applications.convnext import ConvNeXtXLarge
from tensorflow.keras.applications.efficientnet import EfficientNetB7, EfficientNetB3, EfficientNetB4
from collections import Counter
from sklearn.model_selection import StratifiedKFold
import sklearn
from tensorflow.keras.applications.resnet import ResNet152

In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize = False,
                          title = 'Confusion matrix',
                          cmap = plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation = 'nearest', cmap = cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation = 45)
    plt.yticks(tick_marks, classes)
    plt.grid(None)

    if normalize:
        cm = cm.astype('float')/cm.sum(axis = 1)[:, np.newaxis]

    thresh = cm.max()/2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment = "center",
                 color = "white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.show()

def plot_roc_curve(fpr, tpr, label = None):
    """
    The ROC curve, modified from 
    Hands-On Machine learning with Scikit-Learn and TensorFlow; p.91
    """
    plt.figure(figsize = (10, 10))
    plt.title('ROC Curve')
    plt.plot(fpr, tpr, linewidth = 2, label = label)
    plt.plot([0, 1], [0, 1], 'k--')
    plt.axis([-0.005, 1, 0, 1.005])
    plt.xticks(np.arange(0,1, 0.05), rotation = 90)
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate (Recall)")
    plt.show()

In [ ]:
#Fold 1
model_name = 'Ensemble'
i = 0
n_ensemble = i + 1

In [ ]:
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import concatenate
from numpy import argmax

def load_all_models(n_models):
    all_models = list()

    for i in n_models:
        filename = '/kaggle/input/seniortraindata/Model/Model/' + i + '.h5'
        model = load_model(filename)
        all_models.append(model)
        print('>loaded %s' % filename)

    return all_models

n_models = [f'Xception_fold_{n_ensemble}', f'InceptionV3_fold_{n_ensemble}', f'VGG19_fold_{n_ensemble}', f'ResNet152_fold_{n_ensemble}']
models = load_all_models(n_models)
print('Loaded %d models' % len(models))

In [ ]:
def preprocessing_f(no_model):
    if no_model == 1:
        return keras.applications.xception.preprocess_input
    elif no_model == 2:
        return keras.applications.inception_v3.preprocess_input
    elif no_model == 3:
        return keras.applications.vgg19.preprocess_input
    elif no_model == 4:
        return keras.applications.resnet.preprocess_input
    
def generator_type(g_type, no_model):
    if g_type == 'train':
        return ImageDataGenerator(preprocessing_function = preprocessing_f(no_model),
                                     horizontal_flip = True, 
                                     vertical_flip = False, 
                                     height_shift_range = 0.05, 
                                     width_shift_range = 0.02, 
                                     rotation_range = 5, 
                                     shear_range = 0.01,
                                     fill_mode = 'nearest',
                                     zoom_range = 0.05)
    elif g_type == 'valid':
        return ImageDataGenerator(preprocessing_function = preprocessing_f(no_model))

def generate_generator_multiple(g_type, dataframe):
    
    genX1 = generator_type(g_type, 1).flow_from_dataframe(dataframe = dataframe,
                                         directory = None,
                                         x_col = 'Image_name',
                                         y_col = 'Label',
                                         class_mode = 'sparse',
                                         classes = ['0', '1'],
                                         shuffle = True if g_type == 'train' else False,
                                         target_size = (img_height, img_width),
                                         color_mode = 'rgb',
                                         batch_size = batch_size,
                                         seed = 42)
    
    genX2 = generator_type(g_type, 2).flow_from_dataframe(dataframe = dataframe,
                                         directory = None,
                                         x_col = 'Image_name',
                                         y_col = 'Label',
                                         class_mode = 'sparse',
                                         classes = ['0', '1'],
                                         shuffle = True if g_type == 'train' else False,
                                         target_size = (img_height, img_width),
                                         color_mode = 'rgb',
                                         batch_size = batch_size,
                                         seed = 42)
    
    genX3 = generator_type(g_type, 3).flow_from_dataframe(dataframe = dataframe,
                                         directory = None,
                                         x_col = 'Image_name',
                                         y_col = 'Label',
                                         class_mode = 'sparse',
                                         classes = ['0', '1'],
                                         shuffle = True if g_type == 'train' else False,
                                         target_size = (img_height, img_width),
                                         color_mode = 'rgb',
                                         batch_size = batch_size,
                                         seed = 42)
    
    genX4 = generator_type(g_type, 4).flow_from_dataframe(dataframe = dataframe,
                                         directory = None,
                                         x_col = 'Image_name',
                                         y_col = 'Label',
                                         class_mode = 'sparse',
                                         classes = ['0', '1'],
                                         shuffle = True if g_type == 'train' else False,
                                         target_size = (img_height, img_width),
                                         color_mode = 'rgb',
                                         batch_size = batch_size,
                                         seed = 42)
    
    while True:
        X1i = genX1.next()
        X2i = genX2.next()
        X3i = genX3.next()
        X4i = genX4.next()
        yield [X1i[0], X2i[0], X3i[0], X4i[0]], X2i[1]

In [ ]:
def define_stacked_model(members):

    #update all layers in all models to not be trainable
    for i in range(len(members)):
        model = members[i]
        for layer in model.layers:
            layer.trainable = False
            layer._name = 'ensemble_' + str(i+1) + '_' + layer.name

    ensemble_visible = [model.input for model in members]
    ensemble_outputs = [model.output for model in members]
    merge = concatenate(ensemble_outputs)
    x = Dense(32, activation ='relu', kernel_regularizer = regularizers.l2(0.0001))(merge)
    x = Dropout(0.3)(x)
    output = Dense(2, activation ='softmax')(x)

    model = Model(inputs=ensemble_visible, outputs=output)
    plot_model(model, show_shapes=True, to_file='model_graph.png')
    model.compile(optimizer = Adam(lr = 0.0001, decay = 1e-5), loss = "sparse_categorical_crossentropy", metrics = ['accuracy'])

    return model

model = define_stacked_model(models)

In [ ]:
red_lr = ReduceLROnPlateau(monitor = 'val_accuracy', patience = 5, verbose = 1,factor = 0.1)
early_stop = EarlyStopping(monitor = 'val_accuracy', mode = 'max', patience = 10, restore_best_weights = True)

In [ ]:
# Training with K-fold cross validation
kf = StratifiedKFold(n_splits = 5, random_state = 42, shuffle = True)
X = np.array(df['Image_name'])
Y = np.array(df['Label'])

fold = []
train = []
test = []

for fold_no, (train_index, test_index) in enumerate(kf.split(X, Y)):
    print(f"Fold {fold_no+1}:")
    print(f"  Train: index = {train_index}")
    print(f"  Test:  index = {test_index}")
    fold.append(fold_no+1)
    train.append(train_index)
    test.append(test_index)

In [ ]:
trainData = X[train[i]]
testData = X[test[i]]
## create train, valid dataframe and thus train_gen , valid_gen for each fold-loop
train_df = df.loc[df['Image_name'].isin(list(trainData))]
valid_df = df.loc[df['Image_name'].isin(list(testData))]
    
train_gen = generate_generator_multiple('train', train_df)
valid_gen = generate_generator_multiple('valid', valid_df)

print(f'Training for fold {fold[i]} ...')
    
train_class_weights = sklearn.utils.class_weight.compute_class_weight(
                                            class_weight = 'balanced',
                                            classes = ['0','1'], 
                                            y = train_df['Label'].values.tolist())
class_weight = dict(enumerate(train_class_weights))
print(class_weight)

History = model.fit(
            train_gen,
            steps_per_epoch = len(train_df)//batch_size,
            epochs = epochs,
            validation_data = valid_gen,
            validation_steps = len(valid_df)//batch_size,
            class_weight = class_weight,
            callbacks = [red_lr, early_stop])

model.save(f'/kaggle/working/{model_name}_fold_{i+1}.h5')

l, ac = model.evaluate(valid_gen, steps = len(valid_df)//batch_size)

print('model name: ', model_name) 
print(f'Score for fold {fold[i]}: Loss = {l}; Accuracy = {ac}')


plt.figure(figsize = (10, 10))
plt.plot(History.history['loss'])
plt.plot(History.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epochs')
plt.legend(['train', 'test'])
plt.show()

plt.figure(figsize = (10, 10))
plt.plot(History.history['accuracy'])
plt.plot(History.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epochs')
plt.legend(['train', 'test'])
plt.show()